---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import numpy as np

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
all_numbers_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
all_numbers_2 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
all_numbers = pd.concat([all_numbers_1,all_numbers_2])
all_numbers.reset_index(inplace=True)
all_numbers.head()

,level_0,match,0,1,2
0,0,0,03,25,93
1,1,0,6,18,85
2,2,0,7,8,71
3,3,0,9,27,75
4,4,0,2,6,96


In [3]:
month_day_year = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[- .]*)((?:\d{1,2}[, -]*)\d{4})')
month_day_year.reset_index(inplace=True)
month_day_year.head()

,level_0,match,0,1
0,194,0,April,"11, 1990"
1,195,0,May,"30, 2001"
2,196,0,Feb,"18, 1994"
3,197,0,February,"18, 1981"
4,198,0,October.,"11, 2013"


In [4]:
day_month_year = df.str.extractall(r'(\d{1,2}[, -.]*)?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[, -.]*)(\d{4})')
day_month_year.reset_index(inplace=True)
day_month_year.head()

,level_0,match,0,1,2
0,125,0,24,Jan,2001
1,126,0,10,Sep,2004
2,127,0,26,May,1982
3,128,0,28,June,2002
4,129,0,06,May,1972


In [5]:
month_year_numbers = df.str.extractall(r'(\d{1,2})[/](\d{4})')
month_year_numbers.reset_index(inplace=True)
a=[]
for i in month_year_numbers.level_0: 
    if not(i in all_numbers.level_0.values): 
        a.append(i)
a= np.asarray(a)
month_year_numbers = month_year_numbers[month_year_numbers['level_0'].isin(a)]
month_year_numbers.head()

,level_0,match,0,1
25,343,0,6,1998
26,344,0,6,2005
27,345,0,10,1973
28,346,0,9,2005
29,347,0,03,1980


In [6]:
year2 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
year1 = df.str.extractall(r'(^\d{4})')
year = pd.concat([year2,year1])
year.reset_index(inplace=True)
a=[]
for i in year.level_0:
    if not((i in all_numbers.level_0.values) | (i in month_year_numbers.level_0.values) | (i in day_month_year.level_0.values) | (i in month_day_year.level_0.values)):
        a.append(i)
a=np.asarray(a)
year = year[year['level_0'].isin(a)]
year.head()

,level_0,match,0
356,455,0,1984
357,456,0,2000
358,457,0,2001
359,458,0,1982
360,459,0,1998


In [7]:
all_numbers.columns = ['level_0', 'match', 'month', 'day', 'year']
all_numbers['year']=all_numbers['year'].apply(lambda x: '19'+ x if len(x)<=2 else x)
all_numbers.year

0      1993
1      1985
2      1971
3      1975
4      1996
5      1979
6      1978
7      1989
8      1986
9      1971
10     1985
11     1975
12     1998
13     1972
14     1982
15     1998
16     1977
17     1998
18     1979
19     1990
20     1976
21     1982
22     1989
23     1976
24     1971
25     1986
26     1972
27     1987
28     1991
29     1983
       ... 
95     1998
96     1996
97     1990
98     1977
99     1996
100    1990
101    2011
102    1976
103    1984
104    1985
105    2012
106    1986
107    2002
108    1998
109    1987
110    2010
111    1982
112    1994
113    1985
114    1999
115    1974
116    2000
117    1981
118    2004
119    1990
120    1974
121    2002
122    2006
123    2011
124    2006
Name: year, Length: 125, dtype: object

In [8]:
month_day_year[1] = month_day_year[1].replace(',','',regex=True)
month_day_year['day'], month_day_year['year'] = month_day_year[1].str.split(' ', 1).str
month_day_year.drop([1], axis=1, inplace=True)
month_day_year.columns = ['level_0', 'match', 'month', 'day', 'year']
month_day_year.head()

,level_0,match,month,day,year
0,194,0,April,11,1990
1,195,0,May,30,2001
2,196,0,Feb,18,1994
3,197,0,February,18,1981
4,198,0,October.,11,2013


In [9]:
day_month_year.columns = ['level_0', 'match', 'day', 'month', 'year']
day_month_year.day.replace(np.nan, 1, inplace = True)
day_month_year['month'] = day_month_year.month.apply(lambda x: x[:3])
day_month_year['month'] = pd.to_datetime(day_month_year.month, format='%b').dt.month
day_month_year.tail(10)

,level_0,match,day,month,year
174,333,0,1,11,1997
175,334,0,1,7,1986
176,335,0,1,2,1973
177,336,0,1,3,1978
178,337,0,1,12,2007
179,338,0,1,4,1998
180,339,0,1,3,2005
181,340,0,1,5,1980
182,341,0,1,11,2007
183,342,0,1,3,1976


In [10]:
month_year_numbers.columns = ['level_0', 'match', 'month', 'year']
month_year_numbers['day'] = 1
month_year_numbers.head()

,level_0,match,month,year,day
25,343,0,6,1998,1
26,344,0,6,2005,1
27,345,0,10,1973,1
28,346,0,9,2005,1
29,347,0,03,1980,1


In [11]:
year.columns = ['level_0', 'match', 'year']
year['day'] = 1
year['month'] = 1
year.head()

,level_0,match,year,day,month
356,455,0,1984,1,1
357,456,0,2000,1,1
358,457,0,2001,1,1
359,458,0,1982,1,1
360,459,0,1998,1,1


In [12]:
all_dates = pd.concat([all_numbers, month_day_year, day_month_year, month_year_numbers, year])
all_dates['date'] =pd.to_datetime(all_dates['month'].apply(str)+'/'+all_dates['day'].apply(str)+'/'+all_dates['year'].apply(str))
all_dates = all_dates.sort_values('date')
all_dates.reset_index(inplace=True)
all_dates.level_0

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     415
17     335
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    243
472    208
473    139
474    320
475    383
476    480
477    286
478    244
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64

In [13]:
def date_sorter():
    
    return all_dates.level_0